In [1]:
import argparse
from timeit import default_timer as timer
from collections import Counter, defaultdict
import csv
import hashlib

from nltk import tokenize
import numpy as np
import pandas as pd

def preprocess(text):
    # Tokenize
    tokenizer = tokenize.RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)

    # Normalize
    return [token.lower() for token in tokens]

def hash_term(term):
    return int(hashlib.md5(term.encode()).hexdigest(), 16) % (10**8)

In [2]:
inverted_index = defaultdict(set)

with open('./datasets/vkmarco-docs.tsv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter='\t')

    for i, text in enumerate(reader):
        tokens = preprocess(f'{text[2]}. {text[3]}')
        doc = text[0]
        
        for token in tokens:
            hashed_token = hash_term(token)
            inverted_index[hashed_token].add(doc)

In [3]:
with open(f'inverted_index.txt', 'w', encoding='utf-8') as file:
    for word, docs_id in inverted_index.items():
        file.write(f'{word}: {sorted(docs_id)}\n')

In [4]:
with open(f'inverted_index.txt', 'r', encoding='utf-8') as file:
    file_lines = file.readlines()

In [5]:
inverted_index = {}

for line in file_lines:
    word, doc_ids = line.split(':')
    doc_ids_cleaned = doc_ids.strip()[1:-1].replace(' ', '').replace('\'', '').split(',')
    inverted_index[int(word)] = list(doc_ids_cleaned)
    #inverted_index[word] = list(doc_ids_cleaned)

In [8]:
query_doc = {}

with open('./datasets/objects.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    next(reader)  # Пропускаем заголовок
    for parts in reader:
        key = int(parts[0])
        value = parts[1:]
        query_doc[key] = value

In [164]:
with open(f'./datasets/vkmarco-doceval-queries.tsv', 'r', encoding='utf-8') as file:
    doceval_queries = list(csv.reader(file, delimiter='\t'))

In [10]:
num_docs = 9025


result_matrix = np.zeros((len(doceval_queries) * num_docs, 2))
result_matrix[:, 0] = np.arange(1, len(doceval_queries) * num_docs + 1)


for query in doceval_queries:
    print(query)
    docs_ids = []
    tokens = preprocess(query[1])
    num_query = query[0]

    for token in tokens[1:]:
        hashed_token = hash_term(token)
        if hashed_token in inverted_index:
            docs_ids.append(set(inverted_index[hashed_token]))

    if len(docs_ids) > 0:
        result = docs_ids[0]
        for doc_id in docs_ids[1:]:
            result = result.intersection(doc_id)

        objects_id = [query_doc[(num_query, doc)] for doc in result]
        if len(objects_id) > 0:
            result_matrix[np.array(objects_id) - 1, 1] = 1

NameError: name 'doceval_queries' is not defined

In [170]:
import pandas as pd
submission = pd.DataFrame(result_matrix, columns=["ObjectId", "Label"])
submission["Label"] = submission["Label"].fillna(0).astype(int)
submission['ObjectId'] = submission['ObjectId'].astype(int)
submission.to_csv("submission.csv", index=False)

In [171]:
submission[submission['Label'] == 1]

,ObjectId,Label
79,80,1
1744,1745,1
4564,4565,1
4888,4889,1
7741,7742,1
...,...,...
900913,900914,1
901078,901079,1
901328,901329,1
901887,901888,1
